In [36]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine

In [37]:
# Load CSV data into DataFrame
new_passenger_data = pd.read_csv('titanic passenger list.csv')

In [38]:
engine = create_engine('sqlite:///titanic.sqlite', echo=False)

In [39]:
new_passenger_data.to_sql('new_passenger', con=engine, if_exists='replace', index=True)

1309

In [40]:
# Using pandas to inspect column names in the new_passenger table
df_new_passenger = pd.read_sql_table('new_passenger', con=engine)
print(df_new_passenger.columns)

Index(['index', 'pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


In [41]:
# Using pandas to inspect column names in the new_passenger table
df_passenger = pd.read_sql_table('passenger', con=engine)
print(df_passenger.columns)

Index(['id', 'name', 'pclass', 'age', 'sex', 'survived'], dtype='object')


In [42]:
# SQL statement for merging tables and creating a new table
sql = """
CREATE TABLE passenger_merge AS
SELECT p.*, np."home.dest", np.body, np.fare
FROM passenger AS p
JOIN new_passenger AS np
ON p.name = np.name;
"""

# Execute the SQL command
with engine.connect() as connection:
    connection.execute(text(sql))

OperationalError: (sqlite3.OperationalError) table passenger_merge already exists
[SQL: 
CREATE TABLE passenger_merge AS
SELECT p.*, np."home.dest", np.body, np.fare
FROM passenger AS p
JOIN new_passenger AS np
ON p.name = np.name;
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [43]:
df_merged = pd.read_sql('passenger_merge', con=engine)
print(df_merged.head())

Empty DataFrame
Columns: [id, name, pclass, age, sex, survived, home.dest, body, fare]
Index: []


In [44]:
test_sql = """
SELECT COUNT(*)
FROM passenger AS p
JOIN new_passenger AS np
ON p.name = np.name;
"""

with engine.connect() as connection:
    result = connection.execute(text(test_sql))
    print(result.scalar())  # This prints the count of matching rows

0
